In [ ]:
!az ml model show --name Llama-2-7b --version 20 --registry-name azureml-meta

In [ ]:
id="azureml://registries/azureml-meta/models/Llama-2-7b/versions/20"

In [ ]:
%%writefile workspace_policy.json
{
  "properties": {
    "displayName": "[Preview]: Azure Machine Learning Model Registry Deployments are restricted except for the allowed Registry",
    "policyType": "BuiltIn",
    "mode": "Microsoft.MachineLearningServices.v2.Data",
    "description": "Only deploy Registry Models in the allowed Registry and that are not restricted.",
    "metadata": {
      "version": "1.0.0-preview",
      "category": "Machine Learning",
      "preview": true
    },
    "version": "1.0.0-preview",
    "parameters": {
      "allowedRegistryName": {
        "type": "String",
        "metadata": {
          "displayName": "Allowed Registry Name",
          "description": "The exact registry name where models are allowed to be deployed. If not set, by default all models from all registries will be restricted."
        },
        "defaultValue": ""
      },
      "restrictedModels": {
        "type": "Array",
        "metadata": {
          "displayName": "Restricted Model AssetIds",
          "description": "The list of Model AssetIds from the allowedRegistryName to be restricted from being deployed. If not set, by default all models from the allowedRegistryName will be allowed."
        },
        "defaultValue": []
      },
      "effect": {
        "type": "String",
        "metadata": {
          "displayName": "Effect",
          "description": "'Deny' restricts deploying the registry models except for the allowedRegistryName."
        },
        "allowedValues": [
          "Deny",
          "Disabled"
        ]
      }
    },
    "policyRule": {
      "if": {
        "allOf": [
          {
            "field": "type",
            "equals": "Microsoft.MachineLearningServices.v2.Data/workspaces/deployments"
          },
          {
            "value": "[resourceGroup().name]",
            "like": "*antonslutsky-rg"
          },
          {
            "anyOf": [
              {
                "field": "Microsoft.MachineLearningServices.v2.Data/workspaces/deployments/model.assetId",
                "in": "[parameters('restrictedModels')]"
              },
              {
                "allOf": [
                  {
                    "field": "Microsoft.MachineLearningServices.v2.Data/workspaces/deployments/model.assetId",
                    "like": "azureml://registries/*"
                  },
                  {
                    "not": {
                      "field": "Microsoft.MachineLearningServices.v2.Data/workspaces/deployments/model.assetId",
                      "like": "[concat('azureml://registries/', parameters('allowedRegistryName'), '/*')]"
                    }
                  }
                ]
              }
            ]
          }
        ]
      },
      "then": {
        "effect": "[parameters('effect')]"
      }
    },
    "versions": [
      "1.0.0-preview"
    ]
  },
  "id": "/providers/Microsoft.Authorization/policyDefinitions/19539b54-c61e-4196-9a38-67598701be90",
  "type": "Microsoft.Authorization/policyDefinitions",
  "name": "19539b54-c61e-4196-9a38-67598701be90"
}